In [1]:
# Bibliotecas além do gerenciador Anaconda
!pip install spacy
!python -m spacy download pt_core_news_sm
!pip install wordcloud
!pip install gensim
!pip install tensorflow

2022-05-26 13:01:57.773780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-26 13:01:57.774893: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



[+] Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [2]:
# Módulos básicos para manuseio de dados e arquivos
import numpy as np
import pandas as pd
import os
from os.path import isfile, join
import nltk
import re
import string
import unicodedata
import json

# Módulos para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud
%matplotlib inline

# Módulo para processamento de linguagem
import spacy

## Carregamento de textos

In [3]:
limited_news_path = r'Software\Fake.br-Corpus' #\fake_10 or \true_10
news_path = r'Software\Fake.br-Corpus\full_texts' #\fake or \true

paths = [limited_news_path, news_path]

In [18]:
def sortDir(dir_path: str, is_meta=False) ->list:
    '''
    Ordena os arquivos dentro de dir_path e os retorna no formato de lista.
    '''
    if is_meta:
        number_separator = "-meta.txt"
    else:
        number_separator = ".txt"

    first_list = os.listdir(dir_path)
    int_list = [int(element.split(number_separator)[0]) for element in first_list]
    int_list.sort()
    final_list = [(str(element) + number_separator) for element in int_list]

    return final_list

def txtToDataframe(path, is_limited=True):
    '''
    Function for converting full texts to a single DataFrame.
    '''
    if is_limited:
        true_files = [path+"\\true_10\\"+f for f in sortDir(dir_path = path+'\\true_10') if isfile(join(path+'\\true_10', f))]
        fake_files = [path+"\\fake_10\\"+f for f in sortDir(dir_path = path+'\\fake_10') if isfile(join(path+'\\fake_10', f))]
    else:
        true_files = [path+"\\true\\"+f for f in sortDir(dir_path = path+'\\true') if isfile(join(path+'\\true', f))]
        fake_files = [path+"\\fake\\"+f for f in sortDir(dir_path = path+'\\fake') if isfile(join(path+'\\fake', f))]
    
    texts = []
    labels = []
    
    for file in true_files:
        with open(file, encoding='utf-8-sig') as f:
            texts.append(f.read())
            labels.append(0)
    for file in fake_files:
        with open(file, encoding='utf-8-sig') as f:
            texts.append(f.read())
            labels.append(1)
            
    df = pd.DataFrame(list(zip(texts,labels)),columns=['texts','labels'])
    
    # Com esta função, textos e labels foram inseridos em um DataFrame de maneira sequencial. Todas as notícias verdadeiras vêm
    # ANTES do bloco de notícias falsas.
    
    return df

def appendMetadata(path,df, is_limited=True):
    '''
    Function for appending metadata to previously generated news DataFrame.
    '''
    if is_limited:
        true_meta = [path+"\\true-meta-information-10\\"+f for f in sortDir(dir_path = path+'\\true-meta-information-10',is_meta=True) if isfile(join(path+'\\true-meta-information-10', f))]
        fake_meta = [path+"\\fake-meta-information-10\\"+f for f in sortDir(dir_path = path+'\\fake-meta-information-10',is_meta=True) if isfile(join(path+'\\fake-meta-information-10', f))]
    else:
        true_meta = [path+"\\true-meta-information\\"+f for f in sortDir(dir_path = path+'\\true-meta-information',is_meta=True) if isfile(join(path+'\\true-meta-information', f))]
        fake_meta = [path+"\\fake-meta-information\\"+f for f in sortDir(dir_path = path+'\\fake-meta-information',is_meta=True) if isfile(join(path+'\\fake-meta-information', f))]
    

    #true_meta e fake_meta são listas com todas os paths para arquivos de metadata.
    
    columns = ["author", "source", "category", "date","tokens","words_without_punctuation","types","number_of_links","uppercase_words","verbs","subjuntive_imperative","nouns","adjectives","adverbs","modal_verbs","singular_first_and_second_personal_pronouns","plural_first_personal_pronouns","pronouns","pausality","characters","avg_sentence_length","avg_word_length","percentage_of_spelling_errors","emotiveness","diversity"]
    
    true_metadata = pd.DataFrame(columns=columns)
    fake_metadata = pd.DataFrame(columns=columns)
    
    for file in true_meta:
        #print(file)
        aux = pd.read_csv(file, header=None, sep = '\n').transpose()
        aux.columns = columns
        true_metadata=true_metadata.append(aux)
        
        
    for file in fake_meta:
        #print(file)
        aux = pd.read_csv(file, header=None, sep = '\n').transpose()
        aux.columns = columns
        fake_metadata=fake_metadata.append(aux)
        
    
    metadata = pd.DataFrame(columns=columns)
    metadata = metadata.append(true_metadata,ignore_index=True)
    metadata = metadata.append(fake_metadata,ignore_index=True) 


    complete_df = pd.concat([df,metadata],axis=1)
    # Este DataFrame possui todos os textos/labels (2 colunas) e metadata (25 colunas).
    
    return complete_df

In [19]:
ai = int(input('''0 - Base com 10 notícias verdadeiras e 10 notícias falsas
1 - Base completa de notícias
'''))

path = paths[ai]

if ai == 0:
    data = txtToDataframe(path) # Dataframe contendo notícias e labels.
    complete_data = appendMetadata(path,data) # Dataframe contendo notícias, labels e metadata.
else:
    data = txtToDataframe(path,is_limited=False)
    complete_data = appendMetadata(path,data,is_limited=False)

0 - Base com 10 notícias verdadeiras e 10 notícias falsas
1 - Base completa de notícias
0


In [20]:
complete_data['texts'][14]

'Temer resolve o problema de Luislinda: \x93liberdade, ainda que tardia\x94.\n\nA ministra dos Direitos Humanos Luislinda Valois entrou para a galeria dos \x93sem noção\x94 ao pedir para acumular o salário da função de ministra com a aposentadoria de desembargadora, o que daria R$ 61 mil, muito além do teto constitucional. Ao fazê-lo, alegou que sua condição, \x93sem sombra de dúvida, se assemelha a trabalho escravo\x94.\n\nAgora o jornal carioca O Dia divulga que Temer decidiu demitir Luislinda e dar um fim ao seu problema. Ao que parece, o presidente ainda busca uma saída honrosa para Luislinda. Talvez outro cargo. Não sabemos ainda.\n\nSua substituta deve ser a deputada federal licenciada Tia Eron (PRB), atual secretária municipal de Promoção Social.\n\nO que importa é que Luislinda, enfim, conseguiu sua liberdade. Ainda que tardia.'

## Preprocessamento de textos

In [11]:
# Carregando o pacote de língua portuguesa para o processador Spacy
nlp = spacy.load('pt_core_news_sm')

In [10]:
# Defininido funções de preprocessamento

def removePunct(text):
    '''
    Removes any punctuation included in string.punctuation.
    '''
    translator = text.maketrans({key:'' for key in string.punctuation+'“”'}) # Translates any punctuation into ''
    return text.translate(translator)

def removeNumbers(text):
    '''
    Removes any number character in text.
    '''
    return re.sub('[0-9]', '' , text) # Translates any number into ''

def removeStopWords(string):
    '''
    Removes any portuguese stopwords, using Spacy's standard package.
    '''
    doc = nlp(string)
    return ' '.join([token.text for token in doc if token.is_stop is False])

def lemmatize(string):
    '''
    Lemmatizes text word-by-word. Notice that lemmatizing is not as harsh as stemming, which makes the final text easier to read and understand in common language.
    '''
    doc = nlp(string)
    return ' '.join([token.lemma_ for token in doc])

def prep(string, useStopWords = False, lemma = False):
    '''
    Executes previously defined preprocessing in text.
    '''

    result = removeNumbers(removePunct(string)).lower()
    
    if useStopWords and lemma:
        doc = nlp(result)
        result = ' '.join([token.lemma_ for token in doc if token.is_stop is False])
    elif useStopWords:
        doc = nlp(result)
        result = ' '.join([token.text for token in doc if token.is_stop is False])
    elif lemma:
        doc = nlp(result)
        result = ' '.join([token.lemma_ for token in doc])

    result = result.replace('\n',"")
    
    return result

In [12]:
# Realizando preprocessamento de textos presentes no Dataframe de notícias completo.

complete_data['texts'] = complete_data['texts'].apply(prep)

In [14]:
complete_data['texts'][15]

'moro está certo ao não se arrepender de divulgação de áudios de lula e dilma ele virou um heróia revista veja promoveu o fórum amarelas ao vivo que contou com a presença do juiz federal sérgio morono evento ele foi foi indagado sobre a divulgação do áudio entre lula e dilma em março do ano passado assim ele respondeu  \x93não me arrependo de forma nenhuma embora tenha ficado consternado com a celeuma que a divulgação causou\x94\x93na minha opinião eu fiz o que a lei exigia e o que eu achei que era necessário\x94 disse \x93não eram exatamente conversas republicanas\x94 disse \x93não cabe ao poder judiciário servir guardião dos segredos sombrios dos nossos governantes\x94 completoupara quem não se lembra na época lula e dilma conversavam ao telefone sobre a indicação do expresidente ao ministério da casa civilo objetivo estava mais do que óbvio dilma o indicou para lhe conceder foro privilegiado em clara obstrução da justiçafoi a partir da revelação destes áudios que se criou o clima pa

In [59]:
# Assignando variáveis dependentes e independentes

y = complete_data['labels'].values # y is strings for labels; but should be fake-0/true-1
X = [d.split() for d in complete_data['texts'].tolist()] # X is a list of lists of words.

In [ ]:
print(X[14])

## Tokenization (TensorFlow)

In [10]:
# Importando módulos para tokenização de textos
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [60]:
# Assignando bases de teste e treino

test_limit = 0.10
training_sentences,testing_sentences,training_labels,testing_labels = train_test_split(X,y,test_size=test_limit, random_state=42)

#training_sentences = X[0:test_limit]
#testing_sentences = X[test_limit:]

#training_labels = y[0:test_limit]
#testing_labels = y[test_limit:]

In [ ]:
type(training_sequences)

In [12]:
# Gerando dicionário de tokens (com base nos textos de treinamento)

tokenizer = Tokenizer()#oov_token='<OOV>')

tokenizer.fit_on_texts(training_sentences)

In [13]:
tokenizer.word_index

{'de': 1,
 'a': 2,
 'o': 3,
 'que': 4,
 'e': 5,
 'do': 6,
 'da': 7,
 'em': 8,
 'para': 9,
 'no': 10,
 'um': 11,
 'com': 12,
 'não': 13,
 'na': 14,
 'é': 15,
 'uma': 16,
 'os': 17,
 'por': 18,
 'se': 19,
 'foi': 20,
 'as': 21,
 'ao': 22,
 'dos': 23,
 'como': 24,
 'mais': 25,
 'ele': 26,
 'à': 27,
 'mas': 28,
 'pelo': 29,
 'das': 30,
 'lula': 31,
 'ser': 32,
 'também': 33,
 'são': 34,
 'presidente': 35,
 'disse': 36,
 'pela': 37,
 'tem': 38,
 'sobre': 39,
 'está': 40,
 'anos': 41,
 'ou': 42,
 'eu': 43,
 'já': 44,
 'federal': 45,
 'sua': 46,
 'governo': 47,
 'ainda': 48,
 'isso': 49,
 'r': 50,
 'seu': 51,
 'segundo': 52,
 'brasil': 53,
 'entre': 54,
 'temer': 55,
 'até': 56,
 'nos': 57,
 'foram': 58,
 'quando': 59,
 'ter': 60,
 'há': 61,
 'diz': 62,
 'ela': 63,
 'era': 64,
 'país': 65,
 'contra': 66,
 'expresidente': 67,
 'muito': 68,
 '–': 69,
 'pessoas': 70,
 'dia': 71,
 'nas': 72,
 'vai': 73,
 'pode': 74,
 'ano': 75,
 'após': 76,
 'mesmo': 77,
 'caso': 78,
 'sem': 79,
 'afirmou': 80,
 

In [14]:
# Realizando a sequencialização das bases de treinamento e teste

training_sequences = tokenizer.texts_to_sequences(training_sentences)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

In [18]:
# Analisar notícias maiores do que max_length

min_length = 100
max_length = 100

#plt.hist([len(x) for x in X], bins = 750)
#plt.show()

nos = np.array([len(x) for x in training_sentences])
print("There are "+ str(len(nos[nos>=max_length])) + " news equal or longer than "+ str(max_length) + " words (will be truncated).")

nos = np.array([len(x) for x in training_sentences])
print("There are "+ str(len(nos[nos<min_length])) + " news shorter than "+ str(min_length) + " words (will be killed).")

There are 5877 news equal or longer than 100 words (will be truncated).
There are 603 news shorter than 100 words (will be killed).


In [19]:
# Matar notícias menores do que min_length

flag_dict = {}
for i in range(0,len(training_sequences)):
    if len(training_sequences[i]) < min_length:
        flag_dict[i] = True
    else:
        flag_dict[i] = False
        
training_labels = [training_labels[i] for i in range(0,len(flag_dict)) if (flag_dict[i] == False)]

training_sequences = [seq for seq in training_sequences if len(seq)>=min_length]

In [20]:
for i in range(0,len(training_sequences)):    
    print(len(training_sequences[i]))

1986
1506
269
219
595
558
691
305
176
219
244
490
454
191
944
156
112
387
274
1128
1469
2549
235
681
744
1060
133
361
162
279
135
144
278
607
723
722
1741
119
1451
2092
1850
477
277
1601
539
950
160
130
2115
1352
114
284
130
411
1837
182
151
186
180
608
765
611
293
391
629
143
722
578
1901
252
333
261
317
142
1122
229
391
890
782
288
839
1649
1275
123
437
128
219
115
232
838
163
1834
1010
2348
140
114
388
287
177
1597
2469
140
674
761
1723
1124
121
115
1211
244
1067
214
1096
394
201
123
1890
131
236
196
1463
129
328
235
118
1562
115
685
977
371
1916
143
166
673
347
1112
134
141
767
1647
1081
935
757
179
1239
216
2001
557
180
611
224
134
592
606
175
284
946
135
168
1326
140
527
803
264
811
686
915
1492
680
222
186
660
177
150
332
1885
685
1017
1162
497
125
111
335
925
108
1277
179
145
168
142
497
616
255
135
113
111
325
639
865
848
1466
772
2226
557
3064
774
646
207
243
934
744
995
1088
1134
109
555
115
111
300
503
1187
1172
241
749
1199
183
253
371
885
919
2362
1649
163
648
911
380
394

1936
1164
1563
896
399
633
1371
120
856
138
1542
897
858
753
643
129
2406
113
274
740
268
159
757
122
1207
144
318
218
101
225
103
584
481
879
124
1052
451
822
694
220
200
247
661
417
121
776
576
132
122
152
539
1553
651
243
422
1398
394
174
467
1401
144
121
274
223
2096
501
149
173
157
176
194
230
1718
237
114
829
195
177
168
139
1466
128
851
269
224
136
106
239
212
508
628
124
182
327
156
250
332
846
151
908
1521
473
592
652
509
528
405
355
275
1141
654
3067
460
801
171
141
515
1599
636
1060
661
920
162
303
2071
230
110
232
454
700
610
164
186
126
102
409
865
965
1134
106
1460
599
727
1790
215
119
1308
384
257
114
1769
957
2225
428
502
408
737
1058
707
724
129
419
1372
223
750
2217
825
425
197
1634
280
154
225
1192
263
859
1364
200
823
1184
1892
300
2724
1960
222
188
105
211
1469
1032
1587
762
1543
181
247
1063
112
119
1115
199
119
106
1330
190
229
421
411
595
198
132
953
119
1195
182
758
2058
801
1126
103
807
213
708
129
203
1490
1952
999
275
897
1255
239
946
583
627
561
1005
1866
1

2235
686
532
433
1998
1218
1206
255
182
144
166
378
715
688
131
398
131
203
825
640
146
178
827
804
767
1112
459
134
684
773
275
140
1434
719
186
1127
870
474
822
1714
1235
302
204
1784
1991
1939
1440
209
1191
206
1990
402
109
2051
380
127
263
114
646
2938
226
1003
100
232
169
105
398
110
185
1459
181
1397
1373
169
1228
577
387
995
1206
190
625
109
2411
485
145
102
887
128
123
129
1166
160
797
172
979
1491
616
293
193
126
293
193
473
1696
566
269
292
1306
208
668
680
698
591
668
116
583
119
223
180
228
881
238
199
1854
635
165
122
1745
161
2772
1464
201
371
2033
1318
136
187
126
135
290
384
1557
945
1295
135
624
607
133
917
1010
156
371
158
112
771
986
142
354
126
169
330
453
309
143
574
165
1837
778
169
900
1007
589
862
575
103
117
126
425
278
198
1442
1302
2172
631
304
899
567
853
392
903
238
464
1306
867
1908
503
135
484
142
952
1829
623
4468
671
178
477
637
532
207
461
2297
108
666
188
1556
874
208
240
243
125
589
125
169
1074
194
131
193
531
181
1317
148
2040
900
1592
151
175
1332

In [21]:
# Truncar (padding) das sequências tokenizadas

training_padded = pad_sequences(training_sequences,maxlen=max_length,padding='post',truncating='pre')
#testing_padded = pad_sequences(testing_sequences,maxlen=max_length,padding='post',truncating='pre')

In [22]:
for i in range(0,len(training_padded)):    
    print(len(training_padded[i]))

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100


100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100


100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100


In [23]:
vocab = tokenizer.word_index

In [ ]:

inv_vocab = {v: k for k, v in vocab.items()} # reconverte o token para a palavra correpondente.

training_texts = []
testing_texts = []

# Com estas listas "_texts", temos os textos novamente completos, mas agora com 'OOV's para palavras desconhecidas
# na base de testes.
for sequence in training_sequences:
    news=[]
    for token in sequence:
        news.append(inv_vocab[token])  #inv_vocab[token] é a palavra correspondente ao token.
    training_texts.append(news)


for sequence in testing_sequences:
    news=[]
    for token in sequence:
        news.append(inv_vocab[token])
    testing_texts.append(news)

In [ ]:
testing_texts[0]

## Vectorization (gensim)

In [24]:
import gensim

In [25]:
DIM = 100
w2v_model = gensim.models.Word2Vec(sentences=training_sentences, vector_size=DIM, window=10, min_count=1)

In [26]:
w2v_model.wv.most_similar('bolsonaro')

[('jair', 0.907778799533844),
 ('ciro', 0.8144818544387817),
 ('presidenciável', 0.7822350859642029),
 ('pscrj', 0.7752283811569214),
 ('revidada', 0.7503700852394104),
 ('wyllys', 0.7460529208183289),
 ('jean', 0.7391692399978638),
 ('alckmin', 0.7235313653945923),
 ('psl', 0.7209506034851074),
 ('boulos', 0.7071114182472229)]

In [27]:
w2v_model.wv['bolsonaro']

array([ 4.15374711e-02,  3.22328329e-01,  3.85684460e-01,  1.37886643e+00,
        1.24810183e+00,  1.63449779e-01, -8.32580566e-01,  1.06310606e-01,
        2.18334174e+00, -1.88506141e-01, -1.19094896e+00,  2.11220354e-01,
        3.24106932e+00, -7.81780541e-01, -5.59793591e-01, -2.81305790e+00,
        9.93528545e-01, -6.09950781e-01, -8.70736539e-01, -1.21936512e+00,
        1.62259305e+00,  1.65118825e+00, -2.86780167e+00,  2.42640352e+00,
       -2.17995214e+00,  3.88573349e-01,  6.30785525e-01,  1.58533442e+00,
       -8.45292136e-02,  1.48755682e+00,  3.95757053e-03, -4.67656642e-01,
       -2.70475030e-01, -4.96726990e+00, -1.73591304e+00, -8.36376131e-01,
        4.42570627e-01, -1.66960025e+00, -6.70342520e-02, -2.07076216e+00,
        1.11627555e+00,  2.06510425e+00, -2.47242141e+00, -3.30444306e-01,
        3.33363973e-02, -1.20089710e+00,  1.89976823e+00,  2.91248411e-01,
       -2.43416357e+00, -2.18604732e+00, -1.81491983e+00,  5.35315096e-01,
       -1.17294586e+00,  

In [28]:
vocab_size = len(tokenizer.word_index)+1

def get_weight_matrix(model):
    ''' Inserts every word vector in a NumPy array, ordering them by token icon (which is determined by their appearing frequencies). '''
    weight_matrix = np.zeros((vocab_size, DIM))

    for word, token in vocab.items():
        weight_matrix[token] = model.wv[word]

    return weight_matrix

embedding_vectors = get_weight_matrix(model=w2v_model)

## Criação da rede neural

In [29]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [30]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=DIM, weights = [embedding_vectors], input_length=max_length, trainable=False))
model.add(LSTM(units=128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          9235900   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 9,353,277
Trainable params: 117,377
Non-trainable params: 9,235,900
_________________________________________________________________


In [31]:
model.fit(training_padded, np.transpose(np.asarray(training_labels)), validation_split=0.3, epochs=10)

Epoch 1/10
129/129 [==============================] - 32s 195ms/step - loss: 0.6290 - acc: 0.6394 - val_loss: 0.5756 - val_acc: 0.7075
Epoch 2/10
129/129 [==============================] - 22s 173ms/step - loss: 0.5190 - acc: 0.7471 - val_loss: 0.5399 - val_acc: 0.7415
Epoch 3/10
129/129 [==============================] - 23s 177ms/step - loss: 0.4438 - acc: 0.8014 - val_loss: 0.5015 - val_acc: 0.7585
Epoch 4/10
129/129 [==============================] - 26s 200ms/step - loss: 0.3670 - acc: 0.8432 - val_loss: 0.5014 - val_acc: 0.7681
Epoch 5/10
129/129 [==============================] - 23s 180ms/step - loss: 0.3031 - acc: 0.8755 - val_loss: 0.4912 - val_acc: 0.7732
Epoch 6/10
129/129 [==============================] - 24s 190ms/step - loss: 0.2194 - acc: 0.9173 - val_loss: 0.5140 - val_acc: 0.7891
Epoch 7/10
129/129 [==============================] - 21s 159ms/step - loss: 0.1463 - acc: 0.9521 - val_loss: 0.5710 - val_acc: 0.7681
Epoch 8/10
129/129 [==============================] - 2

In [32]:
sequences = tokenizer.texts_to_sequences(testing_sentences) # Sequencing testing sentences.
padded = pad_sequences(testing_sequences,maxlen=max_length,padding='post',truncating='pre')

In [33]:
y_pred = (model.predict(padded) >=0.5).astype(int)

accuracy_score(testing_labels, y_pred)

0.8125

In [39]:
testing_labels

array([0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,

## Salvando o modelo

In [34]:
def saveModel(model,dicto):
    name = input('nome do modelo:')
    model.save(str(name)+'.h5')
    with open('wordIndex.json','w') as f:
        json.dump(dicto,f,indent = "")
    print('Model saved!')
    return

In [37]:
saveModel(model, vocab)

nome do modelo:0526_rnn_truncado100_acc81
Model saved!


## Buscando notícias

In [73]:
testing_labels[14]

'1Fake'

In [74]:
testing_sentences[14]

['quatro',
 'expresidentes',
 'presos',
 'até',
 'é',
 'o',
 'que',
 'a',
 'lava',
 'jato',
 'está',
 'preparando',
 'a',
 'força',
 'tarefa',
 'da',
 'operação',
 'lava',
 'jato',
 'está',
 'com',
 'farto',
 'material',
 'acusatório',
 'para',
 'levar',
 'para',
 'a',
 'cadeia',
 'simultaneamente',
 'expresidentes',
 'do',
 'brasil',
 'lula',
 'dilma',
 'temer',
 'e',
 'collor',
 'podem',
 'terminar',
 'sua',
 'vida',
 'política',
 'atrás',
 'das',
 'gradesa',
 'condenação',
 'de',
 'lula',
 'acendeu',
 'o',
 'alerta',
 'vermelho',
 'em',
 'políticos',
 'que',
 'se',
 'achavam',
 'intocáveis',
 'a',
 'operação',
 'lava',
 'jato',
 'ao',
 'condenar',
 'o',
 'maior',
 'líder',
 'político',
 'da',
 'esquerda',
 'no',
 'brasil',
 'demonstrou',
 'que',
 'nenhuma',
 'pessoa',
 'está',
 'acima',
 'da',
 'leia',
 'condenação',
 'a',
 'nove',
 'anos',
 'e',
 'seis',
 'meses',
 'de',
 'prisão',
 'pelo',
 'juiz',
 'federal',
 'sergio',
 'moro',
 'nesta',
 'quartafeira',
 'não',
 'mandará',
 'o',